Utilizad la librería BeautifulSoup. 


Vais a tener que pensar artimañas para poder coger la información de todas las páginas del catálogo, de todos los libros...

https://books.toscrape.com/index.html

Titulo
Categoría (fantasía, poesía... etc)
Precio
Si está o no en stock
Cuantos quedan
Portada del libro
Descripción
Información variada: UPC, tipo de producto, precio con impuestos, precio sin impuestos, impuestos, reviews


In [9]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
from IPython.display import display, HTML

# Definir la URL de la página inicial
url_base = "https://books.toscrape.com/catalogue/page-{}.html"
url_base_libro = "https://books.toscrape.com/catalogue/"

# Función para obtener los enlaces de cada libro en una página del catálogo
def obtener_enlaces_libros(pagina):
    url = url_base.format(pagina)
    respuesta = req.get(url)
    sopa = bs(respuesta.content, 'html.parser')
    
    # Encontrar todos los enlaces a los libros
    enlaces_libros = []
    for h3 in sopa.find_all('h3'):
        enlace_libro = h3.find('a')['href']
        enlaces_libros.append(url_base_libro + enlace_libro.replace('../../../', ''))
    return enlaces_libros

# Función para extraer la información de un libro específico
def extraer_info_libro(url_libro):
    respuesta = req.get(url_libro)
    sopa = bs(respuesta.content, 'html.parser')
    
    # Extraer título
    titulo = sopa.find('h1').text
    
    # Extraer categoría
    categoria = sopa.find('ul', class_='breadcrumb').find_all('li')[2].text.strip()
    
    # Extraer precio
    precio = sopa.find('p', class_='price_color').text.strip()
    
    # Extraer disponibilidad y cantidad
    disponibilidad = sopa.find('p', class_='instock availability').text.strip()
    en_stock = "In stock" in disponibilidad
    cantidad = disponibilidad.split('(')[1].split(' ')[0] if en_stock else "0"
    
    # Extraer imagen de la portada
    url_imagen = url_base_libro + sopa.find('img')['src'].replace('../../', '')
    
    # Extraer información variada (UPC, precio con impuestos, precio sin impuestos, etc.)
    info_producto = sopa.find('table', class_='table table-striped')
    info_dict = {}
    for fila in info_producto.find_all('tr'):
        clave = fila.find('th').text.strip()
        valor = fila.find('td').text.strip()
        info_dict[clave] = valor

    # Retornar toda la información como un diccionario
    return {
        "Título": titulo,
        "Categoría": categoria,
        "Precio": precio,
        "En Stock": en_stock,
        "Cantidad": cantidad,
        "URL Imagen": f'<img src="{url_imagen}" width="100">',
        **info_dict
    }

# Función principal para hacer scraping de todas las páginas del catálogo
def hacer_scraping_todos_libros():
    datos_libros = []
    pagina = 1
    while True:
        enlaces_libros = obtener_enlaces_libros(pagina)
        if not enlaces_libros:
            break
        for enlace_libro in enlaces_libros:
            info_libro = extraer_info_libro(enlace_libro)
            datos_libros.append(info_libro)
        pagina += 1
    
    # Convertir los datos a un DataFrame de pandas
    df_libros = pd.DataFrame(datos_libros)

    # Convertir columnas de precios a valores numéricos (eliminando el símbolo de libra)
    df_libros['Precio'] = df_libros['Precio'].replace({'£': ''}, regex=True).astype(float)
    df_libros['Price (excl. tax)'] = df_libros['Price (excl. tax)'].replace({'£': ''}, regex=True).astype(float)
    df_libros['Price (incl. tax)'] = df_libros['Price (incl. tax)'].replace({'£': ''}, regex=True).astype(float)

    # Retornar el DataFrame
    return df_libros

# Ejecutar el scraper y guardar los resultados en un DataFrame
df = hacer_scraping_todos_libros()

# Mostrar el DataFrame con las imágenes
display(HTML(df.to_html(escape=False)))


,Título,Categoría,Precio,En Stock,Cantidad,URL Imagen,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews
0,A Light in the Attic,Poetry,51.77,True,22,,a897fe39b1053632,Books,51.77,51.77,£0.00,In stock (22 available),0
1,Tipping the Velvet,Historical Fiction,53.74,True,20,,90fa61229261140a,Books,53.74,53.74,£0.00,In stock (20 available),0
2,Soumission,Fiction,50.10,True,20,,6957f44c3847a760,Books,50.10,50.10,£0.00,In stock (20 available),0
3,Sharp Objects,Mystery,47.82,True,20,,e00eb4fd7b871a48,Books,47.82,47.82,£0.00,In stock (20 available),0
4,Sapiens: A Brief History of Humankind,History,54.23,True,20,,4165285e1663650f,Books,54.23,54.23,£0.00,In stock (20 available),0
5,The Requiem Red,Young Adult,22.65,True,19,,f77dbf2323deb740,Books,22.65,22.65,£0.00,In stock (19 available),0
6,The Dirty Little Secrets of Getting Your Dream Job,Business,33.34,True,19,,2597b5a345f45e1b,Books,33.34,33.34,£0.00,In stock (19 available),0
7,"The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull",Default,17.93,True,19,,e72a5dfc7e9267b2,Books,17.93,17.93,£0.00,In stock (19 available),0
8,The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics,Default,22.60,True,19,,e10e1e165dc8be4a,Books,22.60,22.60,£0.00,In stock (19 available),0
9,The Black Maria,Poetry,52.15,True,19,,1dfe412b8ac00530,Books,52.15,52.15,£0.00,In stock (19 available),0
